In [0]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#trial with A2_A3 dataset
!unzip -qq '/content/gdrive/My Drive/Deep_Learning_Project/A2_A3'

In [0]:
!ls
!cd A2_A3/
!pwd
!ls

A2_A3  gdrive  sample_data
/content
A2_A3  gdrive  sample_data


In [0]:
import pandas as pd
from pandas import DataFrame
df=pd.read_csv("/content/A2_A3/A2/A2.csv",names=['id','label'])
df1_T=df[0:15]
print(df1_T)
df1_V=df[15:23]
print(df1_V)
df_test = df[23:]
print(df_test)
training_set = pd.read_csv("/content/A2_A3/A2/A2.csv").values
y_target = df_test['label'].values

                  id  label
0   plant001_rgb.png      5
1   plant002_rgb.png      4
2   plant003_rgb.png      3
3   plant004_rgb.png      4
4   plant006_rgb.png      5
5   plant007_rgb.png      7
6   plant008_rgb.png      4
7   plant009_rgb.png      5
8   plant011_rgb.png      6
9   plant012_rgb.png      8
10  plant013_rgb.png      5
11  plant014_rgb.png      7
12  plant016_rgb.png      7
13  plant017_rgb.png      9
14  plant018_rgb.png      8
                  id  label
15  plant019_rgb.png      9
16  plant021_rgb.png      9
17  plant022_rgb.png     12
18  plant023_rgb.png     10
19  plant024_rgb.png      9
20  plant026_rgb.png     11
21  plant027_rgb.png     14
22  plant028_rgb.png     12
                  id  label
23  plant029_rgb.png     12
24  plant031_rgb.png     11
25  plant032_rgb.png     15
26  plant034_rgb.png     12
27  plant036_rgb.png     15
28  plant037_rgb.png     18
29  plant038_rgb.png     15
30  plant039_rgb.png     16


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=df1_T,
        directory='/content/A2_A3/A2/A2_T/A2_Train',
        x_col="id",
        y_col="label",
        shuffle=True,
        target_size=(320, 320),
        batch_size=6,
        class_mode="raw")
valid_generator=train_datagen.flow_from_dataframe(
                dataframe=df1_V,
                directory='/content/A2_A3/A2/A2_T/A2_Val',
                x_col="id",
                y_col="label",
                batch_size=6,
                #seed=49,
                shuffle=True,
                class_mode="raw",
                target_size=(320,320))

Found 15 validated image filenames.
Found 8 validated image filenames.


In [0]:
from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-05-01 13:16:09--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   222MB/s    in 0.4s    

2020-05-01 13:16:10 (222 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [0]:
pre_trained_model = InceptionV3(input_shape = (320, 320, 3), 
                                include_top = False, 
                                weights = None)

In [0]:
pre_trained_model.load_weights(local_weights_file)

In [0]:
pre_trained_model.trainable=True
for layer in pre_trained_model.layers:
    layer.trainable = False

In [0]:
#pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 18, 18, 768)


In [0]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='relu')(x)  
print(x)
model = Model( pre_trained_model.input, x) 
model.summary()
from keras import regularizers
model.compile(optimizer = RMSprop(lr=0.001), 
              loss = 'mse', 
              metrics = ['accuracy'])

Tensor("dense_3/Identity:0", shape=(None, 1), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 320, 320, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 159, 159, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 159, 159, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 159, 159, 32) 0           batch_normalization_94[0][0]     
_______________________________

In [0]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=valid_generator,
                    validation_steps=step_size_valid,
                    #steps_per_epoch=1,
                    epochs=50)

#print(history.history['val_loss'])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
2/2 [==============================] - 5s 3s/step - loss: 39.0000 - accuracy: 0.0000e+00 - val_loss: 113.8333 - val_accuracy: 0.0000e+00
Epoch 2/50
2/2 [==============================] - 5s 3s/step - loss: 29.7500 - accuracy: 0.0000e+00 - val_loss: 120.5000 - val_accuracy: 0.0000e+00
Epoch 3/50
2/2 [==============================] - 5s 2s/step - loss: 34.4167 - accuracy: 0.0000e+00 - val_loss: 111.8333 - val_accuracy: 0.0000e+00
Epoch 4/50
2/2 [==============================] - 5s 3s/step - loss: 38.8333 - accuracy: 0.0000e+00 - val_loss: 105.1667 - val_accuracy: 0.0000e+00
Epoch 5/50
2/2 [==============================] - 5s 3s/step - loss: 32.8333 - accuracy: 0.0000e+00 - val_loss: 127.8333 - val_accuracy: 0.0000e+00
Epoch 6/50
2/2 [==============================] - 5s 2s/step - loss: 33.5000 - accuracy: 0.0000e+00 - val_loss: 120.5000 - val_accuracy: 0.0000e+00
Epoch 7/50
2/2 [====================

In [0]:
print(history.history.keys())
print(history.history['loss'][49])


dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
43.916664123535156


In [0]:
test_generator = train_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory='/content/A2_A3/A2/A2_Test',
        x_col="id",
        y_col="label",
        shuffle=True,
        target_size=(320, 320),
        batch_size=6,
        class_mode="raw")

Found 8 validated image filenames.


In [0]:
#results = model.evaluate(test_generator,steps= 1000, verbose = 1)
#print('test loss, test acc:', results)
TEST_SIZE = 8
pred = model.predict(test_generator, steps=TEST_SIZE, verbose=1)
#predictions = model.predict(test_generator, verbose=1)


1000/1000 [==============================] - 1159s 1s/step - loss: 208.5573 - accuracy: 0.0000e+00
test loss, test acc: [208.5573272705078, 0.0]
8/8 [==============================] - 8s 968ms/step


In [0]:
print(len(pred))

32
